# Simplified AWS Bedrock LLM function calling using SuperAgentX.

### Open Source Github Repository URL: https://github.com/superagentxai/superagentx

## What is SuperAgentX?

SuperAgentX is an advanced agentic AI framework designed to accelerate the development of Artificial General Intelligence (AGI). It provides a powerful, modular, and flexible platform for building autonomous AI agents capable of executing complex tasks with minimal human intervention.

Using the SuperAgentX agentic AI framework, function calling with AWS Bedrock LLMs is simplified.

The below example creates weather as a handler (Tool) and invokes using SuperAgentX's agent.

#### The Below example explains how to invoke Bedrock Converse API using the SuperAgentX framework.

In this example, the SuperAgentX Agent module can automatically invoke a mock weather handler (Tool).


In [ ]:
!pip install superagentx

In [11]:
from superagentx.agent import Agent
from superagentx.engine import Engine
from superagentx.llm import LLMClient
from superagentx.handler.base import BaseHandler
from superagentx.prompt import PromptTemplate
from superagentx.agentxpipe import AgentXPipe

In [18]:
%%html
<style>
table {float:left}
</style>

Although this example leverages Claude 3 Sonnet, Bedrock supports many other models. The full list of models and supported features can be found [here](https://docs.aws.amazon.com/bedrock/latest/userguide/conversation-inference.html). The models are invoked via `bedrock-runtime`.


**Best Practice**: Always set your AWS credentials as environment variables.

```
export AWS_ACCESS_KEY=<<`YOUR_ACCESS_KEY`>>
export AWS_SECRET_KEY=<<`YOUR_ACCESS_SECRET_KEY`>>
export AWS_REGION=<<`AWS_REGION`>>
```

#### LLM Config in SuperAgentX

LLM Configuration Specify ==> llm_config

| Name | Description | Data Type | Required | Example |
|------|-------------|-----------|----------|---------|
|`model`| AWS Bedrock supported [models](https://docs.aws.amazon.com/bedrock/latest/userguide/conversation-inference.html)|str| Yes| 'model': 'anthropic.claude-3-5-sonnet-20240620-v1:0' |
|`llm_type`| LLM type - `bedrock`|str| Yes| 'llm_type':'bedrock'


 

In [12]:
llm_config = {'model': 'anthropic.claude-3-5-sonnet-20240620-v1:0', 'llm_type':'bedrock'}

llm_client: LLMClient = LLMClient(llm_config=llm_config)

### Handler (Tool) definition 

We define `WeatherHandler` as a class where individual handlers (tools) are defined as functions. Note that there is nothing specific to the model used or Bedrock in this definition.

In [13]:
from superagentx.handler.base import BaseHandler
from superagentx.handler.decorators import tool

class WeatherHandler(BaseHandler):
    
    @tool
    async def get_weather(self, city: str, state: str) -> str:
        """
        Generate weather in Celsius or Fahrenheit 

        Args:
            @param city (str): The city name
            @param state (str): The state name

            @return result (Str): Return Fake Weather for the given city & name in Fahrenheit  
        """
        result = f'Weather in {city}, {state} is 70F and clear skies.'
        print(f'Tool result: {result}')
        return result

### Create Prompt Template Object

In [14]:
prompt_template = PromptTemplate()

In [15]:
weather_forecast_engine = Engine(
    handler=WeatherHandler(),
    llm=llm_client,
    prompt_template=prompt_template
)

## Agent attributes

| Attribute                  | Parameter  | Description                                                                                                                                                                                                                                    |
| :------------------------- | :--------- | :--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **Role**                   | `role`     | Defines the agent's function within the SuperAgentX. It determines the kind of activcity the agent is best suited for.                                                                                                                                    |
| **Goal**                   | `goal`     | The individual objective that the agent aims to achieve. It guides the agent's decision-making process.                                                                                                                                        |
| **LLM**              | `llm`| Represents the language model that will run the agent.|
| **Retry Max** *(optional)*       | `max_retry`      | sets the maximum number of iterations an agent can complete before it must provide its best possible answer. Default `5`                                                       |
| **Prompt Template**       | `prompt_template`      | Specifies the prompt format for the agent. |                                                       |
| **Engine**    | `engines`      | A list of engines (or lists of engines) that the engine can utilize in `parallel` or `sequnetial`. This allows for flexibility in processing and task execution based on different capabilities or configurations or or configurations.|

In [16]:
weather_man_agent = Agent(
    goal="Verify the weather results for the given city and state ",
    role="Weather Man",
    llm=llm_client,
    max_retry=2, #Default Max Retry is 5
    prompt_template=prompt_template,
    engines=[weather_forecast_engine],
        )

### Pipe Attributes

| Attribute                  | Parameter  | Description                                                                                                                                                                                                                                    |
| :------------------------- | :--------- | :--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **Name**                   | `name`     | An optional name for the agentxpipe, providing a more friendly reference for display or logging purposes.                                                                                                                                    |
| **Agents**                   | `agents`     | A list of Agent instances (or lists of Agent instances) that are part of this structure. These agents can perform tasks and contribute to achieving the defined goal.         |


In [17]:
pipe = AgentXPipe(
            name='Fake Weather Man',
            agents=[weather_man_agent]
)

In [21]:
result = await pipe.flow(query_instruction="What is the weather like in New York, USA?")

Tool result: Weather in New York, New York is 70F and clear skies.


In [22]:
print(result)

[GoalResult(name='Agent-9c51452a31fd417eb6492a01961e7011', agent_id='9c51452a31fd417eb6492a01961e7011', reason='The output context provides weather information for New York, USA, which matches the query instruction.', result='Weather in New York, New York is 70F and clear skies.', content=None, error=None, is_goal_satisfied=True)]
